In [1]:
import pandas as pd
import numpy as np

foods = pd.read_csv('./전국통합식품영양성분정보(가공식품)표준데이터.csv', sep=',', encoding='ms949')

In [2]:
# foodSet = pd.DataFrame(foods, columns=['식품코드', '식품명', '에너지(kcal)', '탄수화물(g)', '단백질(g)', '지방(g)', '포화지방산(g)', '나트륨(mg)'])
foodSet = pd.DataFrame(foods, columns=['식품코드', '식품명', '대표식품명', '식품대분류명', '식품중분류명', '식품소분류명', '에너지(kcal)', '탄수화물(g)', '단백질(g)', '지방(g)', '포화지방산(g)', '나트륨(mg)'])

In [3]:
# 추가 : '식품대분류명' = '식용유지류', '잼류', '당류', '장류', '조미식품' 배제
#print(foodSet['식품대분류명'])

foodSet = foodSet.loc[(foodSet['식품대분류명'] != '식용유지류') & \
                      (foodSet['식품대분류명'] != '잼류') & \
                      (foodSet['식품대분류명'] != '당류') & \
                      (foodSet['식품대분류명'] != '장류') & \
                      (foodSet['식품대분류명'] != '조미식품')]
len(foodSet)

45196

In [4]:
# 결측치 제거  : NaN 있을 경우 K Mean 측정에서 문제 발생
foodSet.fillna(0, inplace=True)

In [5]:
from konlpy.tag import Okt
from konlpy.tag import Kkma

import re;

okt = Okt()
kkma = Kkma()

In [6]:
def extractIngredients():
    
    # 식약처 음식 목록에서 유효단어(가령 음식 구성 단어들)만 추출
    ingredients = set()
    
    for foodname in foodSet['식품명'] :
    
        foodWords = okt.nouns(foodname)
        
        for ingre in foodWords:
            ingredients.add(ingre)
        
    return ingredients

In [7]:
ingreList = extractIngredients()

# 불용어 제거
def cleanWords(foodname):
    
    result = ''
    foodWords = okt.nouns(foodname)
    
    for food in foodWords:

        if food in ingreList: # 식약처 음식 목록에서 추출한 구성 단어들외에는 제거
            result += food + ' '
        
    return result.strip()

cleanWords('새송이버섯버터굴소스볶음 간단반찬')

'송이버섯 버터 굴소스 볶음 반찬'

In [8]:
# foodname : 식약처 음식 목록 ex) 영·유아용 이유식_완료기밥흰살생선감자조림밥
# targetFoodname :  만개의 레시피 음식명  ex) 도시락 반찬으로 최고! 스팸감자조림♡
def searchSimilarFood(foodname, targetFoodname):
    
    # 식품명에 "_" 포함시 뒤부분은 token만 식품명으로 추출
    if "_" in foodname:
        foodname = foodname.split("_")[1] # 완료기밥흰살생선감자조림밥

    if "_" in targetFoodname:
        targetFoodname = targetFoodname.split("_")[1] # 
    
    foodnames = set(okt.nouns(foodname))  
    targetFoodnames = set(okt.nouns(targetFoodname)) # 도시락 반찬 최고 스팸 감자 조림

    print("foodname : ", foodname)
    print("targetFoodnames : ", targetFoodnames)
    print("foodnames : ", foodnames)

    print("타겟 음식 token 길이 : ", len(targetFoodnames))
    print("소스 음식 token 길이 : ", len(foodnames))

    foundWords = [] # 발견된 단어들
    
    for food in foodnames:
        
        if food in targetFoodnames:            
            foundWords.append(food)


        for targetFood in targetFoodnames : 
           if targetFood in food:
               foundWords.append(food)
               print("해당 단어 : ", food)

    foundWords = set(foundWords)
    
    print("foundWords : ", foundWords)

    trueCount = len(foundWords)

    if trueCount == 0:
        similarRate = 0
    else:            
        # 소스/타겟 간의 단어수 비교
        # wordCompareCountRate = len(foodnames) / len(targetFoodnames)
        # print("wordCompareCountRate", wordCompareCountRate)        

        # 단어 일치비율
        similarRate = round(trueCount / len(targetFoodnames) * 100, 2)

        print("단어 match count : ", similarRate)
        
        # 일치하는 단어 갯수가 비율이 적으면 배제
        # 최소 비율 50%    
        # if wordCompareCountRate < 0.5:            
        #     similarRate = similarRate * wordCompareCountRate        
        
    return round(similarRate, 2)

targetFoodname = '새송이버섯버터굴소스볶음 간단반찬'
foodname = '영·유아용 이유식_이지밀한우새송이진밥'
searchSimilarFood(foodname, targetFoodname) 

foodname :  이지밀한우새송이진밥
targetFoodnames :  {'볶음', '송이버섯', '버터', '간단', '굴소스', '반찬'}
foodnames :  {'밥', '한우', '지밀', '송이진'}
타겟 음식 token 길이 :  6
소스 음식 token 길이 :  4
foundWords :  set()


0

In [9]:
searchSimilarFood(foodname, foodname) # 100%

foodname :  이지밀한우새송이진밥
targetFoodnames :  {'밥', '한우', '지밀', '송이진'}
foodnames :  {'밥', '한우', '지밀', '송이진'}
타겟 음식 token 길이 :  4
소스 음식 token 길이 :  4
해당 단어 :  밥
해당 단어 :  한우
해당 단어 :  지밀
해당 단어 :  송이진
foundWords :  {'밥', '한우', '지밀', '송이진'}
단어 match count :  100.0


100.0

In [10]:
targetFoodname = '도시락 반찬으로 최고! 스팸감자조림♡'
foodname = '당면_넓은감자당면'
searchSimilarFood(foodname, targetFoodname) 

foodname :  넓은감자당면
targetFoodnames :  {'도시락', '감자', '스팸', '최고', '조림', '반찬'}
foodnames :  {'감자'}
타겟 음식 token 길이 :  6
소스 음식 token 길이 :  1
해당 단어 :  감자
foundWords :  {'감자'}
단어 match count :  16.67


16.67

In [18]:
targetFoodname = '마지막 묵은지 털어 끓인 김치 콩나물국..'
foodname = '기타 조림_감자조림'
searchSimilarFood(foodname, targetFoodname) 

foodname :  감자조림
targetFoodnames :  {'묵은지', '김치', '마지막', '콩나물국'}
foodnames :  {'감자', '조림'}
타겟 음식 token 길이 :  4
소스 음식 token 길이 :  2
foundWords :  set()


0

In [12]:
"_" in "과·채주스_프룻밀토마토"

True

In [13]:
# foodname : 식약처 음식 목록 ex) 영·유아용 이유식_완료기밥흰살생선감자조림밥
# targetFoodname :  만개의 레시피 음식명  ex) 도시락 반찬으로 최고! 스팸감자조림♡
# searchSimilarFood 함수에서 print 로그 부분을 뺀 함수
def searchSimilarFoodNotComment(foodname, targetFoodname):
    
    # 식품명에 "_" 포함시 뒤부분은 token만 식품명으로 추출
    if "_" in foodname:
        foodname = foodname.split("_")[1] # 완료기밥흰살생선감자조림밥

    if "_" in targetFoodname:
        targetFoodname = targetFoodname.split("_")[1] # 
    
    foodnames = set(okt.nouns(foodname))  
    targetFoodnames = set(okt.nouns(targetFoodname)) # 도시락 반찬 최고 스팸 감자 조림

    # print("foodname : ", foodname)
    # print("targetFoodnames : ", targetFoodnames)
    # print("foodnames : ", foodnames)

    # print("타겟 음식 token 길이 : ", len(targetFoodnames))
    # print("소스 음식 token 길이 : ", len(foodnames))

    foundWords = [] # 발견된 단어들
    
    for food in foodnames:
        
        if food in targetFoodnames:            
            foundWords.append(food)


        for targetFood in targetFoodnames : 
           if targetFood in food:
               foundWords.append(food)
               # print("해당 단어 : ", food)

    foundWords = set(foundWords)
    
    # print("foundWords : ", foundWords)

    trueCount = len(foundWords)

    if trueCount == 0:
        similarRate = 0
    else:            
        # 소스/타겟 간의 단어수 비교
        # wordCompareCountRate = len(foodnames) / len(targetFoodnames)
        # print("wordCompareCountRate", wordCompareCountRate)        

        # 단어 일치비율
        similarRate = round(trueCount / len(targetFoodnames) * 100, 2)

        #print("단어 match count : ", similarRate)
        
        # 일치하는 단어 갯수가 비율이 적으면 배제
        # 최소 비율 50%    
        # if wordCompareCountRate < 0.5:            
        #     similarRate = similarRate * wordCompareCountRate        
        
    return round(similarRate, 2)

In [17]:
#targetFoodname = '도시락 반찬으로 최고! 스팸감자조림♡'
targetFoodname = '맑은 감자계란국 끓이는법 아기계란국'

for foodname in foodSet['식품명']:

    similarRate = searchSimilarFoodNotComment(foodname, targetFoodname)

    result = list()
    
    if similarRate > 50:
        print("단어 일치율 : %.2f, 음식 : %s" % (similarRate, foodname))
        result.append(foodname)

    # 검색이 안되면.... 30%대로 조정
    if len(result) == 0:
        if similarRate > 30:
            print("단어 일치율 : %.2f, 음식 : %s" % (similarRate, foodname))
            result.append(foodname)        

result = result.sort(reverse=True) # 내림차순 정렬

#print(len(result))
print(result)

단어 일치율 : 33.33, 음식 : 칼국수_100국산순도토리칼국수
단어 일치율 : 33.33, 음식 : 칼국수_국수가락 생칼국수
단어 일치율 : 33.33, 음식 : 칼국수_오늘좋은 감자생칼국수
단어 일치율 : 33.33, 음식 : 칼국수_감자생칼국수
단어 일치율 : 33.33, 음식 : 칼국수_감자생칼국수
단어 일치율 : 33.33, 음식 : 칼국수_감자생칼국수
단어 일치율 : 33.33, 음식 : 칼국수_감자생칼국수
단어 일치율 : 33.33, 음식 : 칼국수_감자칼국수
단어 일치율 : 33.33, 음식 : 쌀국수_후루룩쌀국수 미역국
단어 일치율 : 33.33, 음식 : 쌀국수_농심쌀국수 소고기장국
단어 일치율 : 33.33, 음식 : 쌀국수_태국팟타이볶음쌀국수
단어 일치율 : 33.33, 음식 : 쌀국수_더 조은 쌀국수 면발의 비법 미분당전용
단어 일치율 : 33.33, 음식 : 메밀국수/냉면국수_풍국면 들기름 막국수
단어 일치율 : 33.33, 음식 : 라면_국내산감자로만든감자라면
단어 일치율 : 33.33, 음식 : 라면_국내산미역을가득담은미역국라면
단어 일치율 : 33.33, 음식 : 국수_풍국쫄깃국수
단어 일치율 : 33.33, 음식 : 국수_복만네콩국수用국수
단어 일치율 : 33.33, 음식 : 국수_봉평촌감자국수
단어 일치율 : 33.33, 음식 : 국수_계란국수
단어 일치율 : 33.33, 음식 : 국수_가야곡쫄깃감자국수
단어 일치율 : 33.33, 음식 : 국수_감자국수
단어 일치율 : 33.33, 음식 : 두부_농협 한국농협두부 국산콩두부
단어 일치율 : 33.33, 음식 : 두부_농협 한국농협두부 국산콩두부
단어 일치율 : 33.33, 음식 : 두부_농협 한국농협두부 국산콩두부 부침용
단어 일치율 : 33.33, 음식 : 두부_농협 한국농협두부 국산콩두부 찌개용
단어 일치율 : 33.33, 음식 : 떡_미정당 국산쌀로 만든 국물떡볶이
단어 일치율 : 33.33, 음식 : 떡_대숲맑은 담양 쌀떡국떡
단어 일치율 : 33.33, 음식 : 

In [16]:
targetFoodname = '마지막 묵은지 털어 끓인 김치 콩나물국..'

for foodname in foodSet['식품명']:

    similarRate = searchSimilarFoodNotComment(foodname, targetFoodname)

    result = list()
    
    if similarRate > 50:
        print("단어 일치율 : %.2f, 음식 : %s" % (similarRate, foodname))
        result.append(foodname)

    # 검색이 안되면.... 30%대로 조정
    if len(result) == 0:
        if similarRate > 30:
            print("단어 일치율 : %.2f, 음식 : %s" % (similarRate, foodname))
            result.append(foodname)        

result = result.sort(reverse=True) # 내림차순 정렬

#print(len(result))
print(result)

단어 일치율 : 50.00, 음식 : 라면_송송썰린김치가푸짐한김치찌개라면
단어 일치율 : 50.00, 음식 : 라면_수미네 묵은지 김치찌개라면
단어 일치율 : 50.00, 음식 : 배추김치_더테이블 숙성배추김치(묵은지)
단어 일치율 : 50.00, 음식 : 기타김치_손김치 갓김치
단어 일치율 : 50.00, 음식 : 기타김치_손김치 총각김치
단어 일치율 : 50.00, 음식 : 기타김치_보스김치찌개용김치
단어 일치율 : 50.00, 음식 : 기타김치_알타리김치/총각김치
None
